## Detecção de Fraudes em Transações Financeiras

### Importando Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

### Importação e Análise Exploratória dos Dados

In [2]:
# Importando dados
df = pd.read_csv('dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'dataset.csv'

In [ ]:
# Visualização geral
df.head()

In [ ]:
# Tamanho do dataset
df.shape

In [ ]:
# Verificando tipo dos dados
df.info()

In [ ]:
# Verificando colunas
df.columns

In [ ]:
# Verificando valores nulos
# Não há valores nulos
df.isna().sum()

In [ ]:
# Verificando valores duplicados
# Não há valores duplicados
df.duplicated().sum()

In [ ]:
# Distribuição de valores da variável isFraud
df['isFraud'].value_counts()

In [ ]:
# Distribuição de valores da variável isFlaggedFraud
df['isFlaggedFraud'].value_counts()

In [ ]:
# Porcentagem de fraudes
f'{round((df['isFraud'].value_counts()[1] / df.shape[0]) * 100,2)}%'

In [ ]:
# Gráfico - tipos de transações
df['type'].value_counts().sort_values().plot(kind='barh', title='Tipos de Transação')
plt.xlabel('Count')
plt.ylabel('ipo de Transação')
plt.show()

In [ ]:
# Gráfico - Total de fraudes por tipo de transações
# A maior parte das fraudes ocorre na transferência ou saque de valores
fraud_by_type = df.groupby('type')['isFraud'].mean().sort_values()
fraud_by_type.plot(kind='barh', title='Fraud Rate by Type')
plt.xlabel('Fraud Rate')
plt.show()

In [ ]:
# Total de fraudes por tipo de transações
fraud_by_type

In [ ]:
# Distribuição dos valores das transações
df['amount'].describe().astype(int)

In [ ]:
# Dada a assimetria da distribuição dos valores, foi aplicada escala logarítmica para visualização gráfica
sns.histplot(np.log1p(df['amount']), bins=100, kde=True)
plt.title('Transaction Amount Distribution (log scale)')
plt.xlabel('Log(Amount + 1)')
plt.show()

In [ ]:
# Avaliando a distribuição de valores em transações legítimas e fraudulentas com valores < 50 mil
# -  De acordo com o boxplot: transações fraudulentas envolvem valores mais altos e têm maior dispersão
sns.boxplot(data=df[df['amount'] < 50000], x='isFraud', y='amount')
plt.title('Amount X Is Fraud (values under 50k)')
plt.show()

### Analisando anomalias em envio e recebimento de saldo

In [ ]:
# Diferença de saldo da conta remetente após envio de valor
df['balanceDiffOrig'] = df['oldbalanceOrg'] - df['newbalanceOrig']

# Diferença de saldo da conta destinatária após recebimento de valor
df['balanceDiffDest'] = df['newbalanceDest'] - df['oldbalanceDest']

In [ ]:
# Verificando anomalia -> aumento de saldo após o envio de valor
(df['balanceDiffOrig'] < 0).sum()

In [ ]:
# Verificando anomalia -> diminuição de saldo após recebimento de valor
(df['balanceDiffDest'] < 0).sum()

In [ ]:
# Verificando fraudes ao longo do tempo
# No geral, não é possível identificar relação tempo x fraudes, podemos descartar a coluna step
frauds_per_step = df[df['isFraud'] == 1]['step'].value_counts().sort_index()
plt.plot(frauds_per_step.index, frauds_per_step.values, label='Frauds per Step')
plt.xlabel('Step(Time)')
plt.ylabel('Number of Frauds')
plt.title('Frauds Over Time')
plt.show()

In [ ]:
# Deletando coluna stp
df.drop(columns='step', inplace=True)

### Analisando comportamento de usuários

In [ ]:
# Top senders
top_senders = df['nameOrig'].value_counts().head(10)
top_senders

In [ ]:
# Top receivers
top_receivers = df['nameDest'].value_counts().head(10)
top_receivers

In [ ]:
# Relação usuários x fraudes
fraud_users = df[df['isFraud'] == 1]['nameOrig'].value_counts().head(10)
fraud_users

In [ ]:
# Distribuição de fraudes de acordo com o tipo de transação
fraud_types = df[df['type'].isin(['TRANSFER', 'CASH_OUT'])]
fraud_types['type'].value_counts()

In [ ]:
# Visualizando a distribuição de fraudes de acordo com o tipo de transação
sns.countplot(data=fraud_types, x='type', hue='isFraud')
plt.title('Fraud Distribution in Transfer and Cash Out')
plt.show()

In [ ]:
#
zero_after_transfer = df[
    (df['oldbalanceOrg'] > 0) &
    (df['newbalanceOrig'] == 0) &
    (df['type'].isin(['TRANSFER', 'CASHOUT']))
]

len(zero_after_transfer)

In [ ]:
zero_after_transfer.head()

In [ ]:
# Matriz de correlação
corr = df[['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'isFraud']].corr()

sns.heatmap(corr, annot=True, fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

### Preparando dados para nosso modelo preditivo

In [ ]:
# Deletando colunas que não serão úteis para nosso modelo
df_model = df.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1)
df_model.head()

In [ ]:
# Separando em variáveis categóricas e numéricas
categorical = ['type']
numeric = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

In [ ]:
# Variável target
y = df_model['isFraud']

# Variáveis preditoras
X = df_model.drop('isFraud', axis=1)

In [ ]:
# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [ ]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), numeric),
        ('cat', OneHotEncoder(drop='first'), categorical)
    ],
    remainder = 'drop'
)

In [ ]:
pipeline = Pipeline([
    ('prep', preprocessor),
    ('clf', LogisticRegression(class_weight='balanced', max_iter=1000))
])

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
pipeline.score(X_test, y_test) * 100

In [ ]:
import joblib

joblib.dump(pipeline, 'fraud_detection_pipeline.pkl')